In [ ]:
RANDOM_STATE = 2025
TEST_PCT = 0.5
SUBSAMPLE_COUNT = None


Path to dataset files: /home/craig-wilkinson/.cache/kagglehub/datasets/agungpambudi/network-malware-detection-connection-analysis/versions/3
             ts                 uid        id.orig_h  id.orig_p  \
0  1.538573e+09   Cu3Tieri43IPsyBO3  192.168.100.113    42789.0   
1  1.538573e+09  Cbetl72NeXczaqQ8Lj  192.168.100.113    60546.0   
2  1.538573e+09  CUmQWl2ZA2hrbNtohe  192.168.100.113    37320.0   
3  1.538573e+09   C9oEZWtG35wx1Mqq5  192.168.100.113    51338.0   
4  1.538573e+09  CbhqpU1oZZyrP7PE6i  192.168.100.113    60088.0   

       id.resp_h  id.resp_p proto service  duration orig_bytes  ...  \
0  192.168.100.1       53.0   udp     dns         -          -  ...   
1  192.168.100.1       53.0   udp     dns  0.001494         90  ...   
2  192.168.100.1       53.0   udp     dns  5.005135         78  ...   
3  192.168.100.1       53.0   udp     dns  0.001483         90  ...   
4  192.168.100.1       53.0   udp     dns  5.038625        117  ...   

  local_resp missed_bytes his

In [ ]:
import kagglehub
import pandas as pd
import numpy as np

# Download latest version
path = kagglehub.dataset_download("agungpambudi/network-malware-detection-connection-analysis")

print("Path to dataset files:", path)


RELOAD_DATA = False
if not RELOAD_DATA:
  try:
    print(df.head())
  except Exception as e:
    print("No dataframe.  Loading data...")
    RELOAD_DATA=True
if RELOAD_DATA:
  dataframes = []
  import os
  for dirname, _, filenames in os.walk(path):
    for index, filename in enumerate(filenames):
      full_path = os.path.join(dirname, filename)
      print(f"Using file: {full_path}")
      dataframes.append(pd.read_csv(full_path, sep ="|"))
  df = pd.concat(dataframes, ignore_index=True)


In [ ]:

SKIPPED_COLUMNS = [
  'ts', 'uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents', 'detailed-label']
ONE_HOT_COLUMNS = ['proto', 'service', 'conn_state', 'local_orig', 'local_resp', 'history', ]
NUMERIC_COLUMNS = [
   'id.orig_p', 'id.resp_p', #??
   'duration', 'orig_bytes',
     'resp_bytes', 'missed_bytes', 
   'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
]
LABEL_COLUMN = ['label']

def process_data(df):
  df = df[ONE_HOT_COLUMNS + NUMERIC_COLUMNS + LABEL_COLUMN]

  if SUBSAMPLE_COUNT:
    df = df.sample(n=SUBSAMPLE_COUNT, random_state=42).copy()
  for col in NUMERIC_COLUMNS:
    df.loc[:, col] = pd.to_numeric(df[col], errors='coerce')
    df.loc[:, col] = df[col].fillna(0)

  df_test = df.sample(frac=TEST_PCT)
  features_test = df_test.drop(columns=LABEL_COLUMN)
  y_test = pd.DataFrame()
  y_test['label'] = np.where(df_test[LABEL_COLUMN[0]] == 'Benign', 1, 0)
  
  df_train = df.drop(df_test.index)
  features_train = df_train.drop(columns=LABEL_COLUMN)
  y_train = pd.DataFrame()
  
  #y_train['label'] = (1 if df_train[LABEL_COLUMN[0]] == 'Benign' else 0)
  y_train['label'] = np.where(df_train[LABEL_COLUMN[0]] == 'Benign', 1, 0)
  
  print(features_train.head())
  print(y_train.head())
  print(features_test.head())
  print(y_test.head())
  return features_train, y_train, features_test, y_test



In [ ]:
df_features_train, df_y_train, df_features_test, df_y_test = process_data(df)


/tmp/ipykernel_262875/42738861.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sampled_df.loc[:, col] = sampled_df[col].fillna(0)


         proto service conn_state local_orig local_resp history  id.orig_p  \
21229140   tcp       -         S0          -          -       S    58968.0   
24816257   tcp       -         S0          -          -       S    39102.0   
19037749   tcp       -         S0          -          -       S    48178.0   
5783878    tcp       -         S0          -          -       S    42564.0   
23373406   tcp       -     RSTOS0          -          -       I    55982.0   

          id.resp_p  duration orig_bytes resp_bytes  missed_bytes  orig_pkts  \
21229140       23.0       0.0        0.0        0.0           0.0        1.0   
24816257       23.0       0.0        0.0        0.0           0.0        1.0   
19037749       23.0  3.140466        0.0        0.0           0.0        3.0   
5783878        23.0  0.000002        0.0        0.0           0.0        2.0   
23373406       80.0  2.688476        0.0        0.0           0.0        2.0   

          orig_ip_bytes  resp_pkts  resp_ip_bytes 

In [ ]:
# @title Question 1 - b
from numpy.random.mtrand import f
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

def train_bagging(df_features_train, df_y_train, max_depth=2):

  preprocessor = make_column_transformer(
      (OneHotEncoder(handle_unknown='ignore'), ONE_HOT_COLUMNS),
      remainder='passthrough'
  )
  tree_classifier = sklearn.tree.DecisionTreeClassifier(max_depth=max_depth)

  bagging_classifier = make_pipeline(
      preprocessor,
      sklearn.ensemble.BaggingClassifier(
          estimator=tree_classifier,
          n_estimators=500,
          max_samples=0.5,
          bootstrap=False,
          n_jobs=-1,
          random_state=RANDOM_STATE)
  )
  bagging_classifier.fit(df_features_train, df_y_train)

  return bagging_classifier

bagging_classifier = train_bagging(df_features_train, df_y_train, max_depth=5)

/home/craig-wilkinson/miniconda3/lib/python3.13/site-packages/sklearn/ensemble/_bagging.py:930: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(bagging_classifier)

def compute_metrics(classifier, df_features, df_y):
  results = classifier.predict(df_features)

  predictions = classifier.predict(df_features)
  
  print(classification_report(df_y, predictions))  


compute_metrics(bagging_classifier, df_features_test, df_y_test)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['proto', 'service',
                                                   'conn_state', 'local_orig',
                                                   'local_resp',
                                                   'history'])])),
                ('baggingclassifier',
                 BaggingClassifier(bootstrap=False,
                                   estimator=DecisionTreeClassifier(max_depth=5),
                                   max_samples=0.5, n_estimators=500, n_jobs=-1,
                                   random_state=2025))])
              precision    recall  f1-score   support

           0       1.00      0.98      0.99    259945
           1       0.97      1.00      0.98 